In [1]:
# # get truncate_to_decimal_places from the utils file
include("../library/utils.jl")

# # 
include("../library/eqn.jl")


using .Utils: truncate_to_decimal_places

# Q1

Given Equation:

$$
e^{-x} -x = 0
$$

Solving the given equation is equivalent to solving this equation:
$$
x = e^{-x}
$$

according to the fixed point method, our function $\phi(x)$ is:

$$
\phi(x) = e^{-x}
$$

In [2]:
function phi(x)
    return exp(-x)
end

ans = Eqn.fixed_point_iteration(phi, 1)
print("Fixed point iteration: ")
println(truncate_to_decimal_places(ans, 4))

Fixed point iteration: 0.5671
